In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import image
# import cv2 as cv
import numpy as np
from glob import glob
import ast
import pandas as pd

import torch

In [ ]:

# path_checkpoint = '/home/pf/pfstaff/projects/andresrobirds/DA_baseline/logs/version_220303_16490/checkpoint_last.pth.tar'

# path_checkpoint = '/scratch/andresro/leon_work/birds/DA_baseline/logs_feb25_replicas/version_220303_20050/checkpoint.pth.tar'

path_checkpoint = '/scratch/andresro/leon_work/birds/DA_baseline/logs_feb25_replicas/version_220303_19580/checkpoint_last.pth.tar'


In [ ]:
checkpoint = torch.load(path_checkpoint, map_location="cpu")

In [ ]:
checkpoint.keys()

In [ ]:
checkpoint['config']

In [ ]:
source = checkpoint['source_centroids'].as_tensor()
target = checkpoint['target_centroids'].as_tensor()

seen_classes = checkpoint['seen_classes']
unseen_classes = checkpoint['unseen_classes']

checkpoint['epoch']

In [ ]:
source.shape

In [ ]:
target.shape

In [ ]:
import torch.nn.functional as F
plt.subplot(221)
plt.title('source seen')
plt.hist(F.linear(source[seen_classes].mean(axis=0), source[seen_classes]).numpy())

plt.subplot(222)
plt.title('source Unseen')
plt.hist(F.linear(source[unseen_classes].mean(axis=0), source[unseen_classes]).numpy())

plt.subplot(223)
plt.title('target Seen')
plt.hist(F.linear(target[seen_classes].mean(axis=0), target[seen_classes]).numpy())

In [ ]:
C = torch.einsum('id,jd->ij', source, source)

_ = plt.hist(C.flatten(), bins=100)



In [ ]:
import torch.nn.functional as F
plt.subplot(221)
plt.title('source all vs all')
C = torch.einsum('id,jd->ij', source, source).view(-1)
_ = plt.hist(C, bins=100)
# _ = plt.hist(angles_source)

plt.subplot(222)
plt.title('source seen vs seen')
C = torch.einsum('id,jd->ij', source[seen_classes], source[seen_classes]).view(-1)
_ = plt.hist(C, bins=100)

plt.subplot(223)
plt.title('source seen vs seen')
C = torch.einsum('id,jd->ij', source[seen_classes], source[seen_classes]).view(-1)
_ = plt.hist(C, bins=100)



In [ ]:

f, axs = plt.subplots(4,4, sharex=True, sharey=True, figsize=(10,10))

vectorsA = [source, source[seen_classes], source[unseen_classes],target[seen_classes]]

names = ['Source','Source Seen','Source Unseen','Target Seen']

for i,A in enumerate(vectorsA):

    axs[i,0].set_ylabel(names[i])
    for j,B in enumerate(vectorsA[:i+1]):
        if i == 3:
            axs[i,j].set_xlabel(names[j])

        ax = axs[i,j]
        # ax.set_title(f'{i}{j}')
        C = torch.einsum('id,jd->ij', A, B).view(-1)

        print(f'{names[i]} \t {names[j]} \t mean (std) \t {C.mean():.3f} ({C.std():.3f})')
        _ = ax.hist(C,bins=50, density=True, alpha = 1.0)
# axs[0,1].set_ylabel('---')
    

In [ ]:
def norm_angle(a,a_min=None, a_max= None):
    if a_min is None:
        a_min = a.min()
    if a_max is None:
        a_max = a.max()
    a = (a+np.abs(a_min) )  / (a_max-a_min)
    a = 2*a  - 1
    a = a*np.pi

    return a



In [ ]:

f, axs = plt.subplots(2,2, sharex=True, sharey=True)


ax = axs[0,0]
ax.set_title('source seen')
# angles_source = F.linear(source[0], source[seen_classes]).numpy()
a = torch.einsum('id,jd->ij', source, source[seen_classes]).view(-1)
a = norm_angle(a)

x1 = np.cos(a)
x2 = np.sin(a)
ax.scatter(x1,x2)

ax = axs[0,1]
ax.set_title('source unseen')
a = torch.einsum('id,jd->ij', source, source[unseen_classes]).view(-1)
a = norm_angle(a)

x1 = np.cos(a)
x2 = np.sin(a)
ax.scatter(x1,x2)


ax = axs[1,0]
ax.set_title('target seen')
a = torch.einsum('id,jd->ij', target, source[seen_classes]).view(-1)
a = norm_angle(a)

x1 = np.cos(a)
x2 = np.sin(a)
ax.scatter(x1,x2)


for ax in axs.flatten():
#     # ax.set_xlim(0,1.1)

    ax.set_xlim(-1.1,1.1)
    ax.set_ylim(-1.1,1.1)
    ax.set_aspect('equal')
#     # ax.axis('off')
#     pass

In [ ]:
import torch.nn.functional as F


plt.hist(F.linear(source[0], source).numpy())